## Analyzing Trade Data with SQL


This notebook uses SQL to explore international trade at US ports.

The data cover April 2022 and were gathered from the International Trade API provided by the Census Bureau [[link]](https://www.census.gov/data/developers/data-sets/international-trade.html). The collected trade figures were saved to a MySQL database, along with descriptive information about ports, trading partners and products. This document is a Jupyter notebook querying the MySQL database. 

The document answers basic questions about international trade at US ports during April 2022:
- Which ports are the largest by total trade value?
- Which products reach at least \$1bn imported via given ports?
- Which trading partners demand at least \$250mm of certain mineral products?
- Which trading partner accounts for the largest share of a given port's total trade?

### Connecting to the database

First let's connect to the database with trade flow figures and descriptive info.

To reiterate, the data itself was collected from the official API then loaded into a MySQL database. This notebook interacts with the database via the package `ipython-sql` [[link]](https://pypi.org/project/ipython-sql/).


In [1]:
%load_ext sql
%sql mysql://user@127.0.0.1:3306/trade
%config SqlMagic.displaycon=False

### Database structure

To start, the query below shows the table structure and data types for the database. 

Trade flow figures themselves are in the tables `imports` and `exports`. These tables specify the dollar value of trade flows, for a certain port, trading partner and product. 

The tables `countries`, `hscodes` and `ports` contain descriptive data for these trade flows. 

Each record in `imports` and `exports` provides the value of trade with a specific country, of a specific good, at a specific US port, during April 2022. 



In [2]:
%%sql
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'trade';


14 rows affected.


table_schema,table_name,column_name,data_type
trade,countries,id,smallint
trade,countries,name,varchar
trade,exports,port_id,smallint
trade,exports,country_id,smallint
trade,exports,hscode,int
trade,exports,value,bigint
trade,hscodes,code,int
trade,hscodes,description,varchar
trade,imports,port_id,smallint
trade,imports,country_id,smallint


### Top 10 Ports by Total Trade Value

To start with a basic question, what were the top 10 US ports by combined import and export trade during April 2022?

The query below shows that the port of Los Angeles saw the largest combined international trade on the month, at $26.9 billion. Trade at the port of LA was primarily inbound, consistent with most of the other top ten ports.

In [3]:
%%sql
WITH imports_agg AS (
    SELECT port_id, 
        SUM(value) AS value 
    FROM imports 
    GROUP BY port_id
),

exports_agg AS (
    SELECT port_id, 
        SUM(value) AS value 
    FROM exports 
    GROUP BY port_id
)

SELECT p.name Port, 
    ROUND((COALESCE(e.value, 0)+COALESCE(i.value, 0))/1e9, 1) 'Total Trade, USD bn', 
    ROUND(i.value/1e9, 1) 'Imports, USD bn', 
    ROUND(e.value/1e9, 1) 'Exports, USD bn'
FROM ports p 
    LEFT OUTER JOIN imports_agg i ON p.id = i.port_id
    LEFT OUTER JOIN exports_agg e ON p.id = e.port_id
ORDER BY 2 DESC
LIMIT 10;

10 rows affected.


Port,"Total Trade, USD bn","Imports, USD bn","Exports, USD bn"
"LOS ANGELES, CA",26.9,24.5,2.3
"CHICAGO, IL",25.9,20.6,5.3
"LAREDO, TX",24.8,15.7,9.1
"NEWARK, NJ",22.4,20.8,1.5
"HOUSTON, TX",21.0,9.1,12.0
"JFK INTERNATIONAL AIRPORT, NY",21.0,12.5,8.5
"NEW ORLEANS, LA",14.8,6.2,8.6
"DETROIT, MI",13.4,6.3,7.1
"LOS ANGELES INTERNATIONAL AIRPORT, CA",12.7,7.4,5.3
"SAVANNAH, GA",12.2,9.3,2.9


### Product Classification using HS Codes

The International Trade figures provided by Census classify products using HS codes [[link]](https://www.trade.gov/harmonized-system-hs-codes). 

The HS code system has a numerical structure: each code is a number corresponding to a category of products. The system features nesting, so that high-level categories are very broad, while nested subcategories are more specific. The numerical system accommodates different levels of specificity by linking broad categories to shorter codes, and narrow categories to longer codes. 

The query below illustrates this nested, numeric code sytem by selecting a few codes linked to cereal grains. The broadest category shown here has two digits -- an HS2 Code. The four digit codes are more specific, and the six digit codes are yet more specific. 

Trade flows in the `imports` and `exports` tables are reported at the six digit level. This is the finest detail currently available in the ports dataset from Census. Based on the nested HS code system, we can aggregate up to broader four-digit or two-digit categories by summing up on shortened six-digit codes.

Note: "NESOI" = "Not elsewhere specified or included".

In [4]:
%%sql 
SELECT hs6.code 'HS6 Code', 
    hs6.description  'HS6 Description', 
    hs4.code 'HS4 Code', 
    hs4.description  'HS4 Description', 
    hs2.code 'HS2 Code', 
    hs2.description  'HS2 Description'
FROM hscodes hs6 
    INNER JOIN hscodes hs4 ON FLOOR(hs6.code/100) = hs4.code
    INNER JOIN hscodes hs2 ON FLOOR(hs4.code/100) = hs2.code
WHERE hs2.description = 'CEREALS'
LIMIT 10;

10 rows affected.


HS6 Code,HS6 Description,HS4 Code,HS4 Description,HS2 Code,HS2 Description
100111,DURUM WHEAT SEED,1001,WHEAT AND MESLIN,10,CEREALS
100119,"DURUM WHEAT, NESOI",1001,WHEAT AND MESLIN,10,CEREALS
100191,"WHEAT AND MESLIN SEED, NESOI",1001,WHEAT AND MESLIN,10,CEREALS
100199,"WHEAT AND MESLIN, NESOI",1001,WHEAT AND MESLIN,10,CEREALS
100210,RYE SEED,1002,RYE,10,CEREALS
100290,"RYE, NESOI",1002,RYE,10,CEREALS
100310,BARLEY SEED,1003,BARLEY,10,CEREALS
100390,"BARLEY, NESOI",1003,BARLEY,10,CEREALS
100410,OATS SEED,1004,OATS,10,CEREALS
100490,"OATS, NESOI",1004,OATS,10,CEREALS


To further illustrate this nested structure, the following query recursively links rows in the `hscodes` table using shortened HS codes. The query produces "trees" that spell out linked product descriptions, from more specific to less specific. 

In [5]:
%%sql
WITH RECURSIVE x (tree,stem,depth) AS (
    SELECT description, 
        FLOOR(code/100) stem, 
        0
    FROM hscodes
    WHERE UPPER(description) LIKE '%DURUM%'
    
    UNION ALL
    
    SELECT CONCAT(x.tree,' >>> ',hsc.description), 
        FLOOR(hsc.code/100) stem, 
        x.depth+1
    FROM hscodes hsc, x
    WHERE x.stem = hsc.code    
)

SELECT tree 'HS6 >>> HS4 >>> HS2'
FROM x
WHERE depth = 2;

2 rows affected.


HS6 >>> HS4 >>> HS2
DURUM WHEAT SEED >>> WHEAT AND MESLIN >>> CEREALS
"DURUM WHEAT, NESOI >>> WHEAT AND MESLIN >>> CEREALS"


### Four-digit Categories with Imports of $1bn+ via California Port Districts


This understanding of HS codes opens up questions. For example, what categories at the 4-digit level saw imports of $1bn or more during April 2022, via ports in or near California? The query below shows such product categories. 

The query above selecting the top 10 ports by throughput in April 2022 shows large contributions from distinct but geographically grouped ports, namely the port of Los Angeles and LAX airport. As such, the query here appeals to *port districts*. These are groupings of ports, that may or may not be located in the same state.

How are ports and port districts identified? Each port has a unique four-digit ID, of which the first two identify a port district.

In [6]:
%%sql
WITH districts AS (
    SELECT DISTINCT FLOOR(id/100) id
    FROM ports p
    WHERE p.name LIKE '%, CA'
),

imports_hs4 AS (
    SELECT FLOOR(i.hscode/100) hs4, 
        ROUND(SUM(i.value/1e9), 3) value
    FROM imports i
        INNER JOIN ports p ON i.port_id = p.id
    WHERE FLOOR(p.id/100) IN (SELECT id FROM districts)
    GROUP BY FLOOR(i.hscode/100)
    HAVING SUM(i.value/1e9) >= 1
)

SELECT imports_hs4.hs4 HS4, 
    hscodes.description Description, 
    imports_hs4.value 'Imports, USD bn'
FROM imports_hs4 INNER JOIN hscodes ON 
imports_hs4.hs4 = hscodes.code
ORDER BY 3 DESC;

9 rows affected.


HS4,Description,"Imports, USD bn"
8703,MOTOR CARS & VEHICLES FOR TRANSPORTING PERSONS,2.594
2709,CRUDE OIL FROM PETROLEUM AND BITUMINOUS MINERALS,2.558
8471,AUTOMATIC DATA PROCESS MACHINES; MAGN READER ETC,2.168
8473,PARTS ETC FOR TYPEWRITERS & OTHER OFFICE MACHINES,1.773
8528,"TV RECVRS, INCL VIDEO MONITORS & PROJECTORS",1.557
8517,PHONE SETS; OTH APPARAT TRANS/RECEP VOICE/IMG/DATA,1.532
8542,"ELECTRONIC INTEGRATED CIRCUITS & MICROASSEMBL, PTS",1.232
8708,PARTS & ACCESS FOR MOTOR VEHICLES (HEAD 8701-8705),1.103
2710,OIL (NOT CRUDE) FROM PETROL & BITUM MINERAL ETC.,1.088


### Trading Partners Demanding Mineral Products


The next query jointly looks at product classification and trading partners. 

A relevant question is: to which trading partners does the US export certain goods? Among broad categories of mineral products, to which trading partners did the US export more than $250 million worth of goods during April 2022?

The query below addresses this question. Among salient product categories at the two-digit level, this query identifies trading partners that imported more than $250 million worth of goods during April 2022.

In [7]:
%%sql
WITH hs2_partner_agg AS (
    SELECT FLOOR(e.hscode/1e4) HS2, 
        hs.description, 
        c.name country, 
        ROUND(SUM(e.value)/1e6,1) value
    FROM exports e,  countries c, hscodes hs
    WHERE e.country_id = c.id
    AND FLOOR(e.hscode/1e4) = hs.code
    AND FLOOR(e.hscode/1e4) between 30 AND 39
    GROUP BY FLOOR(e.hscode/1e4), e.country_id
    HAVING SUM(e.value) > 2.5e8
)

SELECT MAX(CASE WHEN description='PHARMACEUTICAL PRODUCTS'
        THEN country ELSE '' END) AS 'PHARMACEUTICAL PRODUCTS',
    MAX(CASE WHEN description='FERTILIZERS'
        THEN country ELSE '' END) AS 'FERTILIZERS',
    MAX(CASE WHEN description='ESSENTIAL OILS ETC; PERFUMERY, COSMETIC ETC PREPS'
        THEN country ELSE '' END) AS 'ESSENTIAL OILS ETC; PERFUMERY, COSMETIC ETC PREPS',
    MAX(CASE WHEN description='MISCELLANEOUS CHEMICAL PRODUCTS'
        THEN country ELSE '' END) AS 'MISCELLANEOUS CHEMICAL PRODUCTS',
    MAX(CASE WHEN description='PLASTICS AND ARTICLES THEREOF'
        THEN country ELSE '' END) AS 'PLASTICS AND ARTICLES THEREOF'
FROM (
    SELECT description,
        HS2,
        country,
        ROW_NUMBER() OVER (PARTITION BY description ORDER BY country) rn
    FROM hs2_partner_agg
    ) x
GROUP BY rn;

9 rows affected.


PHARMACEUTICAL PRODUCTS,FERTILIZERS,"ESSENTIAL OILS ETC; PERFUMERY, COSMETIC ETC PREPS",MISCELLANEOUS CHEMICAL PRODUCTS,PLASTICS AND ARTICLES THEREOF
Belgium,Canada,Canada,Canada,Belgium
Canada,,,Mexico,Canada
China,,,,China
Germany (Federal Republic of Germany),,,,Mexico
Ireland,,,,
Japan,,,,
Netherlands,,,,
Spain,,,,
United Kingdom,,,,


In [8]:
%%sql
select * from hscodes where code between 30 and 40;

11 rows affected.


code,description
30,PHARMACEUTICAL PRODUCTS
31,FERTILIZERS
32,"TANNING & DYE EXT ETC; DYE, PAINT, PUTTY ETC; INKS"
33,"ESSENTIAL OILS ETC; PERFUMERY, COSMETIC ETC PREPS"
34,"SOAP ETC; WAXES, POLISH ETC; CANDLES; DENTAL PREPS"
35,ALBUMINOIDAL SUBST; MODIFIED STARCH; GLUE; ENZYMES
36,EXPLOSIVES; PYROTECHNICS; MATCHES; PYRO ALLOYS ETC
37,PHOTOGRAPHIC OR CINEMATOGRAPHIC GOODS
38,MISCELLANEOUS CHEMICAL PRODUCTS
39,PLASTICS AND ARTICLES THEREOF


### Largest Trading Partner by Port

Another relevant question is, for each port, for which trading partner are combined imports and exports the greatest during April 2022? The query below answers the question for ports in New York-adjacent districts. 

To produce the results, trade flow values across tables `imports` and `exports` are combined and totaled up by port and by partner. The maximum is taken, and joined with descriptve info from the tables `countries` and `ports`.

To be explicit about port districts, the query returns each port's four-digit ID along with the two-digit port district stem. Port district counts are included, to show the number of ports included in each district grouping. The query includes partner frequency counts and a total row count as well.

In [9]:
%%sql
WITH districts AS (
    SELECT DISTINCT FLOOR(id/100) id
    FROM ports p
    WHERE p.name LIKE '%, NY'
), 

ex_im AS (
    SELECT port_id, 
        country_id, 
        SUM(value) value
    FROM (
        SELECT port_id,
            country_id,
            value
        FROM exports
        WHERE FLOOR(port_id/100) IN (SELECT id FROM districts)
        
        UNION ALL
        
        SELECT port_id,
            country_id,
            value 
        FROM imports
        WHERE FLOOR(port_id/100) IN (SELECT id FROM districts)
    ) x
    GROUP BY port_id, country_id
),

ranked AS (
    SELECT port_id, 
        country_id, 
        value,
        RANK() OVER (PARTITION BY port_id ORDER BY value DESC) rank
    FROM ex_im
),

port_max AS (
    SELECT ranked.port_id, p.name, c.name partner, ranked.value 
    FROM ranked
    INNER JOIN ports p ON ranked.port_id = p.id
    INNER JOIN countries c ON ranked.country_id = c.id
    WHERE ranked.rank = 1
)


SELECT name Name, 
    port_id 'Port ID', 
    FLOOR(port_id/100) 'Port District', 
    COUNT(*) OVER (PARTITION BY FLOOR(port_id/100) ) 'Port District Count',
    partner 'Largest Ex/Im Partner',
    COUNT(*) OVER (PARTITION BY partner) 'Partner Count',
    ROUND(value/1e6, 1) 'Port/Partner Ex/Im Total, USD mm',
    COUNT(*) OVER () 'Total Count'
FROM port_max 
ORDER BY 1;

17 rows affected.


Name,Port ID,Port District,Port District Count,Largest Ex/Im Partner,Partner Count,"Port/Partner Ex/Im Total, USD mm",Total Count
"ALBANY, NY",1002,10,6,Sweden,1,29.6,17
"ALEXANDRIA BAY, NY",708,7,6,Canada,11,1574.8,17
"BUFFALO-NIAGARA FALLS, NY",901,9,5,Canada,11,7429.1,17
"CHAMPLAIN-ROUSES POINT, NY",712,7,6,Canada,11,2407.4,17
"CLAYTON, NY",714,7,6,Canada,11,3.8,17
"JFK INTERNATIONAL AIRPORT, NY",1012,10,6,Switzerland,1,3479.2,17
"MASSENA, NY",704,7,6,Canada,11,64.7,17
"MORRISTOWN AIRPORT, NEWARK, NJ",1081,10,6,Brazil,1,0.8,17
"NEW YORK, NY",1001,10,6,China,2,649.1,17
"NEWARK, NJ",1003,10,6,China,2,3253.6,17
